In [2]:
pip install pulp

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 124.4 MB/s eta 0:00:00


In [4]:
import pulp
import pandas as pd
import numpy as np
import io
from google.colab import files

uploaded = files.upload()
agg = pd.read_csv(io.BytesIO(uploaded['ads_agg.csv']))

Saving ads_agg.csv to ads_agg.csv


In [5]:
# Optimization

TOTAL_BUDGET = 1_000_000
num_groups = len(agg)
prob = pulp.LpProblem("ad_alloc", pulp.LpMaximize)
vars_p = {}

for idx, row in agg.iterrows():
    vars_p[idx] = pulp.LpVariable(
        f"imp_{idx}", lowBound=0, upBound=int(row["ub"]), cat="Integer"
    )

prob += pulp.lpSum(vars_p[idx] * row["conv_per_impr"] for idx, row in agg.iterrows())
prob += pulp.lpSum(vars_p[idx] for idx in vars_p) <= TOTAL_BUDGET
prob.solve(pulp.PULP_CBC_CMD(msg=1))

sol = {idx: int(pulp.value(vars_p[idx])) for idx in vars_p}
agg["allocated_impressions"] = agg.index.map(sol).fillna(0).astype(int)
agg["expected_conversions"] = agg["allocated_impressions"] * agg["conv_per_impr"]

# Improvement calculation

# Baseline conversions = sum of conv_per_impr * equal share of impressions
baseline_impressions_per_group = TOTAL_BUDGET / num_groups
baseline_expected_conversions = (agg["conv_per_impr"] * baseline_impressions_per_group).sum()

optimized_expected_conversions = agg["expected_conversions"].sum()

improvement_absolute = optimized_expected_conversions - baseline_expected_conversions
improvement_percent = (improvement_absolute / baseline_expected_conversions) * 100 if baseline_expected_conversions > 0 else float('inf')

print("\nOptimization Results:")
print(f"Baseline expected conversions: {baseline_expected_conversions:,.2f}")
print(f"Optimized expected conversions: {optimized_expected_conversions:,.2f}")
print(f"Absolute improvement: {improvement_absolute:,.2f}")
print(f"Percentage improvement: {improvement_percent:.2f}%")

# Save results
agg.to_csv("ads_allocation_result.csv", index=False)
print("Saved allocation to ads_allocation_result.csv")


Optimization Results:
Baseline expected conversions: 10,049.33
Optimized expected conversions: 22,943.46
Absolute improvement: 12,894.13
Percentage improvement: 128.31%
Saved allocation to ads_allocation_result.csv
